In [45]:
import torch
import importlib
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
importlib.reload(__import__('helpers'))
from helpers import save_fewshot_results

In [ ]:
CONFIG = {
    'n_way': 3,           # Número de classes por episódio
    'n_shot': 5,          # Exemplos de treino por classe
    'n_query': 15,        # Exemplos de teste por classe
    'n_episodes': 100,    # Número de episódios para avaliação
    'model_name': 'deit_base_distilled_patch16_224',
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

In [24]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   
        std=[0.229, 0.224, 0.225]
    )
])

In [25]:
from torchvision import datasets
from collections import defaultdict

dataset = datasets.ImageFolder(
    root='/content/dataset/ham10000/all',
    transform=transform
)

# mapa classe → índices
class_to_indices = defaultdict(list)
for idx, (_, label) in enumerate(dataset):
    class_to_indices[label].append(idx)


In [26]:
import random
import torch
from torch.utils.data import Subset, DataLoader

def create_episode(
    class_to_indices,
    n_way=5,
    n_shot=10,
    n_query=15
):
    classes = random.sample(list(class_to_indices.keys()), n_way)

    support_idx = []
    query_idx = []

    for c in classes:
        indices = random.sample(
            class_to_indices[c],
            n_shot + n_query
        )
        support_idx += indices[:n_shot]
        query_idx   += indices[n_shot:]

    return support_idx, query_idx, classes


In [27]:
def get_episode_loaders(dataset, support_idx, query_idx):
    support_loader = DataLoader(
        Subset(dataset, support_idx),
        batch_size=len(support_idx),
        shuffle=False
    )

    query_loader = DataLoader(
        Subset(dataset, query_idx),
        batch_size=len(query_idx),
        shuffle=False
    )

    return support_loader, query_loader


In [28]:
def extract_features_vit(loader, model):
    features = []
    labels = []

    model.eval()
    with torch.no_grad():
        for imgs, lbls in loader:
            imgs = imgs.to(next(model.parameters()).device)
            feats = model(imgs)      # saída do DeiT sem head
            features.append(feats)
            labels.append(lbls)

    return torch.cat(features), torch.cat(labels)


In [29]:
import torch.nn.functional as F

def evaluate_episode(
    model,
    dataset,
    class_to_indices,
    device,
    n_way=5,
    n_shot=10,
    n_query=15
):
    support_idx, query_idx, classes = create_episode(
        class_to_indices, n_way, n_shot, n_query
    )

    support_loader, query_loader = get_episode_loaders(
        dataset, support_idx, query_idx
    )

    support_features, support_labels = extract_features_vit(
        support_loader, model
    )
    query_features, query_labels = extract_features_vit(
        query_loader, model
    )

    # normalização
    support_features = F.normalize(support_features, p=2, dim=1)
    query_features   = F.normalize(query_features, p=2, dim=1)

    # remapeia rótulos para [0..n_way-1]
    label_map = {c: i for i, c in enumerate(classes)}
    support_labels = torch.tensor(
        [label_map[int(l)] for l in support_labels],
        device=device
    )
    query_labels = torch.tensor(
        [label_map[int(l)] for l in query_labels],
        device=device
    )

    # protótipos
    prototypes = torch.zeros(n_way, support_features.size(1)).to(device)
    for i in range(n_way):
        prototypes[i] = support_features[support_labels == i].mean(0)
    
    # normalizar todos os protótipos
    prototypes = F.normalize(prototypes, p=2, dim=1)

    # similaridade
    sims = torch.mm(query_features, prototypes.t())
    preds = sims.argmax(dim=1)

    acc = (preds == query_labels).float().mean().item()
    return acc, preds.cpu().numpy(), query_labels.cpu().numpy()


In [ ]:
import timm
import torch

print(CONFIG['device'])
model = timm.create_model(
    CONFIG['model_name'],
    pretrained=True,
    num_classes=0
)

model = model.to(CONFIG['device'])
model.eval()


cuda


VisionTransformerDistilled(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inpl

In [ ]:
import numpy as np

accuracies = []
all_predictions = []
all_labels = []

model.eval()

for ep in range(CONFIG['n_episodes']):
    acc, preds, labels = evaluate_episode(
        model,
        dataset,
        class_to_indices,
        device,
        n_way=CONFIG['n_way'],
        n_shot=CONFIG['n_shot'],
        n_query=CONFIG['n_query']
    )
    accuracies.append(acc)
    all_predictions.extend(preds)
    all_labels.extend(labels)
    print(f"Episódio {ep+1}: {acc*100:.2f}%")

mean_acc = np.mean(accuracies)
std_acc  = np.std(accuracies)

print(f"\nAcurácia final: {mean_acc*100:.2f}% ± {std_acc*100:.2f}%")


Episódio 1: 86.67%
Episódio 2: 44.44%
Episódio 3: 82.22%
Episódio 4: 68.89%
Episódio 5: 77.78%
Episódio 6: 84.44%
Episódio 7: 68.89%
Episódio 8: 62.22%
Episódio 9: 86.67%
Episódio 10: 82.22%
Episódio 11: 68.89%
Episódio 12: 64.44%
Episódio 13: 53.33%
Episódio 14: 64.44%
Episódio 15: 62.22%
Episódio 16: 75.56%
Episódio 17: 66.67%
Episódio 18: 40.00%
Episódio 19: 55.56%
Episódio 20: 71.11%
Episódio 21: 46.67%
Episódio 22: 68.89%
Episódio 23: 53.33%
Episódio 24: 62.22%
Episódio 25: 73.33%
Episódio 26: 53.33%
Episódio 27: 66.67%
Episódio 28: 60.00%
Episódio 29: 62.22%
Episódio 30: 64.44%
Episódio 31: 62.22%
Episódio 32: 57.78%
Episódio 33: 75.56%
Episódio 34: 73.33%
Episódio 35: 62.22%
Episódio 36: 62.22%
Episódio 37: 80.00%
Episódio 38: 64.44%
Episódio 39: 48.89%
Episódio 40: 55.56%
Episódio 41: 66.67%
Episódio 42: 84.44%
Episódio 43: 57.78%
Episódio 44: 64.44%
Episódio 45: 66.67%
Episódio 46: 37.78%
Episódio 47: 51.11%
Episódio 48: 73.33%
Episódio 49: 73.33%
Episódio 50: 57.78%

Acurácia

In [ ]:
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Salvar resultados do experimento
exp_dir = save_fewshot_results(
    experiment_name="deit_prototypical_fewshot",
    model_name=CONFIG['model_name'],
    metric_name="Similaridade Cosine",
    normalization="Normalizacao L2",
    accuracies=accuracies,
    n_way=CONFIG['n_way'],
    n_shot=CONFIG['n_shot'],
    n_query=CONFIG['n_query'],
    n_episodes=CONFIG['n_episodes'],
    device=CONFIG['device'],
    all_predictions=all_predictions,
    all_labels=all_labels
)


Resultados salvos em: /content/drive/MyDrive/pdi/resultados/deit_prototypical_fewshot_3way_5shot_50ep
Configuração: 3-way 5-shot (50 episódios)
Acurácia: 65.07% ± 11.56%
Precision (macro): 65.11%
Recall (macro): 65.07%
F1-Score (macro): 65.04%
Arquivos gerados:
  - config.json
  - report.txt
  - confusion_matrix.png
  - predictions.json
  - accuracy_per_episode.png
  - accuracy_distribution.png
  - accuracy_boxplot.png

